In [ ]:
#! pip install chardet


import folium 
import chardet


import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 


from pandas import Series, DataFrame
from folium import Marker, GeoJson
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
from folium.plugins import MarkerCluster, HeatMap

# Data reading and cleaning

In [ ]:
base = open(r'../../_data/data_dengue_peru.csv','rb').read()
det = chardet.detect(base)
charenc = det['encoding']

# Reading data
df = pd.read_csv(r'../../_data/data_dengue_peru.csv', encoding = charenc)

# Generating Ubigeo as string, cause not all Ubigeo values are of 6 digits
df["Ubigeo_str"]  = df['Ubigeo'].astype(str).str.zfill(6)

# Identifying department
df["Ubigeo_dep"]  = df["Ubigeo_str"].str[0:2].astype(int)

# Identifying province
df["Ubigeo_prov"] = df["Ubigeo_str"].str[0:4].astype(int)

# Capital letter to merge
df.columns = df.columns.str.upper()

# Casos formatting
df["CASOS"] = df["CASOS"].astype(str)
df["CASOS"] = df["CASOS"].replace(',','',regex=True)
df["CASOS"] = df["CASOS"].astype(float)
#df["CASOS"] = df["CASOS"].fillna(0)
#df["CASOS"] = df["CASOS"].astype(int)

# Dataframe pre visualization
df

# Cases by district

In [ ]:
shape_1 = gpd.read_file(r'../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')
shape_1['UBIGEO'] = shape_1['UBIGEO'].astype(str).astype(int)


# General cases by district 2021 data frame 
cases_dist_2021 = df.loc[df['AÑO'] == 2021]

# Cases by district
dist_21 = cases_dist_2021.groupby(['UBIGEO'])[['CASOS']].sum()
dist_21["UBIGEO"] = dist_21.index
dist_21.reset_index(drop=True, inplace=True)
dist_21

# Cases by district georeferenced
cases_dist = pd.merge(shape_1, dist_21, how="outer", on="UBIGEO")

# Plot
cases_dist.plot( column='CASOS', cmap='Reds', 
          figsize=(15, 15),
          linestyle='-',
          edgecolor='gray',
          legend = True,
          missing_kwds={"color": "lightgrey"})

# Cases by province

# Cases by department and year

# Cases by department and 2021 quarters